# Model Kaybı ve Doğruluğunu Grafikle Takip Etme
Bu örnekte:
- Modelin doğruluk ve kayıp değerlerini izleme
- Aşırı öğrenmeyi engellemek için Dropout kullanma
- EarlyStopping ile eğitim sürecini otomatik durdurma
- MNIST veri setiyle görsel sınıflandırma yapılacaktır.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


## MNIST Verisini Yükleme ve Hazırlama

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28) / 255.0
x_test = x_test.reshape(-1, 28*28) / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


## Dropout ve EarlyStopping Kullanılan Model

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dropout(0.3))  # %30 nöron devre dışı
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


## Model Eğitimi ve Takibi

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stop],
                    verbose=1)


## Doğruluk ve Kayıp Grafikleri

In [ ]:
plt.figure(figsize=(12,5))

# Doğruluk grafiği
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim')
plt.plot(history.history['val_accuracy'], label='Doğrulama')
plt.title("Doğruluk")
plt.xlabel("Epoch")
plt.ylabel("Oran")
plt.legend()

# Kayıp grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim')
plt.plot(history.history['val_loss'], label='Doğrulama')
plt.title("Kayıp (Loss)")
plt.xlabel("Epoch")
plt.ylabel("Hata")
plt.legend()

plt.tight_layout()
plt.show()


## Örnek Görsel Tahmini

In [ ]:
index = 20
sample = x_test[index].reshape(1, -1)
prediction = model.predict(sample)

plt.imshow(x_test[index].reshape(28,28), cmap='gray')
plt.title(f"Tahmin: {np.argmax(prediction)}, Gerçek: {np.argmax(y_test[index])}")
plt.axis('off')
plt.show()
